---
title: Scientific project utils
---

## Inspirations

- [DrWatson.jl](https://juliadynamics.github.io/DrWatson.jl/stable/)
- [sacred](https://sacred.readthedocs.io/en/stable/)

In [ ]:
#| default_exp project
#| export
import os
from pathlib import Path
import warnings
from typing import Any, Callable, Dict, List, Tuple, Union
from loguru import logger

In [ ]:
from fastcore.test import test_eq

In [ ]:
#| export
def projectdir():
    try:
        path = Path(os.getenv("PIXI_PROJECT_ROOT"))
    except TypeError:
        path = Path().cwd()
    return path

def datadir():
    return projectdir() / "data"

In [ ]:
#| exporti
def valtostring(val: Any, digits: Union[int, None], sigdigits: int) -> str:
    if isinstance(val, float):
        if digits is not None:
            return f"{val:.{digits}f}"
        elif sigdigits is not None:
            return f"{val:.{sigdigits}g}"
    return str(val)

In [ ]:
#| export
def savename(
    c: Dict,
    prefix: str = "",
    suffix: str = "",
    allowedtypes: Tuple = (int, float, str, bool),
    accesses: List[str] = [],
    ignores: List[str] = [],
    digits: Union[int, None] = None,
    connector: str = "_",
    expand: List[str] = [],
    sigdigits: int = 3,
    val2string: Callable = None,
    sort: bool = True,
    equals: str = "=",
) -> str:
    """
    Create a shorthand name based on the parameters in the dictionary `c`.
    
    Parameters:
        c (Dict): The input dictionary containing the parameters.
        prefix (str): The prefix to start the name with.
        suffix (str): The suffix to end the name with.
        allowedtypes (Tuple): Only values of these types are included.
        accesses (List[str]): Specific keys to include in the name.
        ignores (List[str]): Specific keys to ignore.
        digits (Union[int, None]): Number of decimal digits for rounding floats.
        connector (str): String used to connect key-value pairs.
        expand (List[str]): Keys that will be expanded to their nested savename.
        sigdigits (int): Number of significant digits for rounding floats.
        val_to_string (Callable): Function to convert values to strings.
        sort (bool): Whether to sort the keys alphabetically.
        equals (str): Connector between key and value.

    Returns:
        str: The generated shorthand name.
    """


    if any(sep in prefix for sep in ["/", "\\"]):
        warnings.warn(
            "Path separators in `savename` prefixes may break reproducibility on other OS. "
            "The recommended way is using `os.path.join` with `savename` (e.g. `os.path.join(datadir, savename(prefix, data))`)."
        )

    sigdigits = None if digits is not None else sigdigits
    val2string = val2string or (lambda val: valtostring(val, digits, sigdigits))

    labels = accesses or list(c.keys())
    p = (
        sorted(range(len(labels)), key=lambda k: labels[k])
        if sort
        else range(len(labels))
    )

    first = not prefix or prefix.endswith(os.path.sep)
    s = prefix
    for j in p:
        label = labels[j]
        if label in ignores:
            continue
        val = c[label]
        t = type(val)
        if any(issubclass(t, x) for x in allowedtypes):
            if label in expand:
                if not val:
                    continue
                sname = savename(
                    val,
                    connector=",",
                    digits=digits,
                    sigdigits=sigdigits,
                    equals=equals,
                )
                if not sname:
                    continue
                entry = f"{label}{equals}({sname})"
            else:
                entry = f"{label}{equals}{val2string(val)}"
            if not first:
                s += connector
            s += entry
            first = False

    if suffix:
        s += f".{suffix}"

    return s

In [ ]:
# Examples
d = {"a": 0.153456453, "b": 5, "mode": "double"}
test_eq(savename(d), "a=0.153_b=5_mode=double")
test_eq(savename(d, digits=4), "a=0.1535_b=5_mode=double")
test_eq(savename(d, prefix="n"), "n_a=0.153_b=5_mode=double")
test_eq(savename(d, suffix="n"), "a=0.153_b=5_mode=double.n")
test_eq(savename(d, prefix="n", suffix="n", connector="-"), "n-a=0.153-b=5-mode=double.n")
test_eq(savename(d, allowedtypes=(str,)), "mode=double")
test_eq(savename(d, connector=" | ", equals=" = "), "a = 0.153 | b = 5 | mode = double")

rick = {"never": "gonna", "give": "you", "up": "!"}
test_eq(savename(rick), "give=you_never=gonna_up=!")
test_eq(savename(rick, ignores=["up"]), "give=you_never=gonna")


In [ ]:
#| export
def setup_run_dir(c: Dict, base_dir: Path = datadir(), change_dir: bool = True, **kwargs):
    """
    Create a run directory based on the parameters in the dictionary `c`.

    Parameters:
        c (Dict): The dictionary containing the parameters.
        base_dir (Path): The base directory to create the run directory in.

    Returns:
        Path: The run directory.
    """

    sub_dir = savename(c, **kwargs)
    directory = base_dir / sub_dir
    os.makedirs(directory, exist_ok=True)
    if change_dir:
        os.chdir(directory)
        logger.info(f"Changed directory to {directory}")

In [3]:
# | export
import shutil


def increment_backup_num(filepath: Path):
    if "_" in filepath.stem and filepath.stem.rsplit("_", 1)[-1].isdigit():
        base, num = filepath.stem.rsplit("_", 1)
        new_num = int(num) + 1
    else:
        base, new_num = filepath.stem, 1
    return filepath.with_name(f"{base}_{new_num}{filepath.suffix}")


def safesave(filepath: Path, data, save_func: Callable):
    if filepath.exists():
        backup_path = increment_backup_num(filepath)
        while backup_path.exists():
            backup_path = increment_backup_num(backup_path)
        shutil.move(str(filepath), str(backup_path))
    return save_func(filepath, data)

In [ ]:
# Example usage
data = "Some data to be savedsss"
safesave("test.txt", data)